In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.2


In [2]:
!python -V

Python 3.9.5


In [3]:
import pickle
import pandas as pd

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/ariel/.cache/pypoetry/virtualenvs/mlops-zoomcamp-vQfEKNh3-py3.9/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ariel/.cache/pypoetry/virtualenvs/mlops-zoomcamp-vQfEKNh3-py3.9/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [36]:
year=2023
month=4

In [55]:
input_file=f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
output_file=f"output/yellow_output{year:04d}-{month:02d}.parquet"

In [46]:
df = read_data(input_file)

In [47]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### Answer 1


In [58]:
pd.DataFrame(y_pred).mean()

0    14.292283
dtype: float64

### Answer 2

In [49]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [50]:
df_prediction=pd.DataFrame()

In [51]:
df_prediction['ride_id']=df['ride_id']

In [52]:
df_prediction['prediction_ride']=y_pred

In [53]:
df_prediction.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)